# S3. Score Consistency

Here, we briefly demonstrate the similarities and discrepancies between LIANA's output in python and R. 

In [1]:
import os

import pandas as pd
import scipy

import scanpy as sc

import cell2cell as c2c
from liana.method.sc._rank_aggregate import AggregateClass, _rank_aggregate_meta
from liana.method.sc import cellphonedb, natmi, singlecellsignalr

data_path = '../../data/'
output_folder = os.path.join(data_path, 'liana-outputs/')
c2c.io.directories.create_directory(output_folder)

../../data/liana-outputs/ already exists.


## Comparison with R output:

There are minor differences with the LIANA implementation in R that lead to outputs not being identical

- SingleCellSignalR Magnitude (lrscore): precision - slightly different after 3rd decimal place
- LogFC Specificity (lr_logfc): similar relative differences but different exact values
- CellPhoneDB Specificity (cellphone_pvals): similar relative differences but different exact values
- CellChat: not run by default in R

Let's check the consistency in the magnitude aggregate rank score when running the different methods that report magnitude (excluding CellChat, which is not present by default in R). 

In [6]:
adata = sc.read_h5ad(os.path.join(data_path, 'processed.h5ad'))
sadata = adata[adata.obs['sample']=='C100']

<font color='red'>Note for Daniel - I believe line 198-202 of _liana_pipe.py makes it impossible to pass just "Magnitude" as the consensus_opts argument, so can't pass for example just cellphonedb and natmi and then specify consensus_opts = ['Magnitude']. Have to include singlecellsignalr for this to work (which is fine, still get a high correlation:</font>

In [ ]:
# # make rank_aggregate function that only runs on methods of choice and only for Magnitude
# rank_aggregate_partial = AggregateClass(_rank_aggregate_meta, methods=[cellphonedb, natmi, singlecellsignalr])
# # have to add singlecellsignalr to make the below code work
# rank_aggregate_partial(adata = sadata, 
#                        groupby='celltype', 
#                        use_raw = False, # run on log- and library-normalized counts
#                        verbose = True, 
#                        inplace = True
#                        #consensus_opts = ['Magnitude'] # rank by magnitude only - CURRENTLY not passed to _aggregate
#                       )

In [7]:
# make rank_aggregate function that only runs on methods of choice and only for Magnitude
rank_aggregate_partial = AggregateClass(_rank_aggregate_meta, methods=[cellphonedb, natmi, singlecellsignalr])
rank_aggregate_partial(adata = sadata, 
                       groupby='celltype', 
                       use_raw = False, # run on log- and library-normalized counts
                       verbose = True, 
                       inplace = True
                      )

Using `.X`!
5580 features of mat are empty, they will be removed.
The following cell identities were excluded: Plasma


/home/hratch/miniconda3/envs/ccc_protocols/lib/python3.10/site-packages/pandas/core/indexing.py:1728: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.


0.33 of entities in the resource are missing from the data.
Generating ligand-receptor stats for 2548 samples and 19218 features
Running CellPhoneDB


100%|███████████████████████████████████████| 1000/1000 [00:21<00:00, 46.61it/s]


Running NATMI
Running SingleCellSignalR


/home/hratch/miniconda3/envs/ccc_protocols/lib/python3.10/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.


In [8]:
rel_cols = ['source', 'target', 'ligand_complex', 'receptor_complex', 'magnitude_rank']
liana_aggregate_partial = sadata.uns['liana_res'].loc[:,rel_cols]
liana_aggregate_partial.sort_values(by = ['source', 'target', 'ligand_complex', 'receptor_complex'], inplace = True)
liana_aggregate_partial.to_csv(os.path.join(output_folder, 'magnitude_ranks_python.csv'))
liana_aggregate_partial.head()

,source,target,ligand_complex,receptor_complex,magnitude_rank
67,B,B,ACTR2,ADRB2,0.405892
17,B,B,ADAM17,ITGB1,0.081878
54,B,B,ADAM17,RHBDF2,0.334024
23,B,B,ADAM28,ITGA4,0.403503
8,B,B,APOC2,LRP1,1.000000


Note, to run the correlation, make sure to have run the [companion Python tutorial](../ccc_R/S3_Score_Consistency.ipynb) up to the point where you save the csv named "magnitude_ranks_R.csv". 

In [9]:
# read and format R aggregate rank
lap_R = pd.read_csv(os.path.join(output_folder, 'magnitude_ranks_R.csv'), index_col = 0)
lap_R.columns = ['source', 'target', 'ligand_complex', 'receptor_complex', 'aggregate_rank']

# merge the two scores
la = pd.merge(liana_aggregate_partial, lap_R, on = ['source', 'target', 'ligand_complex', 'receptor_complex'], 
                                                how = 'inner')
sr = scipy.stats.spearmanr(la.magnitude_rank, la.aggregate_rank).statistic
print('The spearman correlation bewteen R and python aggregate magnitude scores is: {:.2f}'.format(sr))

The spearman correlation bewteen R and python aggregate magnitude scores is: 0.98


## Decomposition Consistency between Score Types

We can also check how consistent each score type is after running the full Tensor-cell2cell pipeline. The [CorrIndex](https://doi.org/10.1016/j.sigpro.2022.108457) provides a dissimilarity metric that can compare decompositions of the same rank. 

In [4]:
from collections import defaultdict
from tqdm import tqdm
import itertools

import matplotlib
import numpy as np
import torch

import liana as li
from cell2cell.tensor.metrics import correlation_index

In [5]:
if torch.cuda.is_available():
    import tensorly as tl
    tl.set_backend('pytorch')
    device = 'cuda:0'
else:
    device = 'cpu'

In [6]:
data_folder = '../../data/liana-outputs/'
output_folder = '../../data/tc2c-outputs/'
c2c.io.directories.create_directory(output_folder)

../../data/tc2c-outputs/ already exists.


First, let's load the LIANA scores for each sample and score type:

In [7]:
liana_res = pd.read_csv(data_folder + 'LIANA_by_sample.csv')
sorted_samples = sorted(liana_res['sample_new'].unique())

Next, we can generate and decompose the tensor as in [Tutorial 03](./03-Generate-Tensor.ipynb) and [Tutorial 04](./04-Perform-Tensor-Factorization) respectively, but for each magnitude rank score separately. We will use the rank identified by the consensus magnitude score in Tutorial 04. 

In [8]:
tf_optimization_dict = {'regular': {'runs': 1, 
                                         'tol': 10e-7, 
                                           'n_iter_max': 100}, 
                            'robust': {'runs': 100, 
                                         'tol': 10e-8, 
                                           'n_iter_max': 500}}

def run_tensor_pipeline(liana_res, score_type, rank, 
                       tf_optimization = 'robust'):
    # build tensor
    tensor = li.multi.to_tensor_c2c(liana_res=liana_res, # LIANA's dataframe containing results
                                    sample_key='sample_new', # Column name of the samples
                                    source_key='source', # Column name of the sender cells
                                    target_key='target', # Column name of the receiver cells
                                    ligand_key='ligand_complex', # Column name of the ligands
                                    receptor_key='receptor_complex', # Column name of the receptors
                                    score_key=score_type, # Column name of the communication scores to use
                                    non_negative = True, # set negative values to 0
                                    inverse_fun=lambda x: x, # Transformation function -- don't invert because it's not a rank score
                                    non_expressed_fill=None, # Value to replace missing values with 
                                    how='outer', # What to include across all samples
                                    lr_fill=np.nan, # What to fill missing LRs with 
                                    cell_fill = np.nan, # What to fill missing cell types with 
                                    outer_fraction=1/3., # Fraction of samples as threshold to include cells and LR pairs.
                                    lr_sep='^', # How to separate ligand and receptor names to name LR pair
                                    context_order=sorted_samples, # Order to store the contexts in the tensor
                                    sort_elements=True # Whether sorting alphabetically element names of each tensor dim. Does not apply for context order if context_order is passed.
                                   )
    # get metadata
    element_dict = defaultdict(lambda: 'Unknown')
    context_dict = element_dict.copy()
    context_dict.update({'HC1' : 'Control',
                         'HC2' : 'Control',
                         'HC3' : 'Control',
                         'M1' : 'Moderate COVID-19',
                         'M2' : 'Moderate COVID-19',
                         'M3' : 'Moderate COVID-19',
                         'S1' : 'Severe COVID-19',
                         'S2' : 'Severe COVID-19',
                         'S3' : 'Severe COVID-19',
                         'S4' : 'Severe COVID-19',
                         'S5' : 'Severe COVID-19',
                         'S6' : 'Severe COVID-19',
                        })
    dimensions_dict = [context_dict, None, None, None]
    meta_tensor = c2c.tensor.generate_tensor_metadata(interaction_tensor=tensor,
                                              metadata_dicts=[context_dict, None, None, None],
                                              fill_with_order_elements=True
                                             )
    
    # decompose tensor
    tensor.to_device(device)
    tensor.compute_tensor_factorization(rank=rank,
                                        init='random', # Initialization method of the tensor factorization
                                        svd='numpy_svd', # Type of SVD to use if the initialization is 'svd'
                                        random_state=0, # Random seed for reproducibility
                                        normalize_loadings=True,
                                        runs=tf_optimization_dict[tf_optimization]['runs'],
                                        tol=tf_optimization_dict[tf_optimization]['tol'],
                                        n_iter_max=tf_optimization_dict[tf_optimization]['n_iter_max']
                                       )
    return tensor

In [9]:
tensor_magnitude = c2c.io.read_data.load_tensor(os.path.join(output_folder, 'BALF-Tensor_decomposed.pkl'))
magnitude_scores = ['lr_means', 'expr_prod', 'lrscore', 'lr_probs']

/home/hratch/miniconda3/envs/ccc_protocols/lib/python3.11/site-packages/tensorly/backend/pytorch_backend.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [ ]:
tensors_individual = {}
# calculate on each magnitude score type separately
for score_type in magnitude_scores:
    ti = run_tensor_pipeline(liana_res = liana_res,
                             score_type = score_type,
                             rank = tensor_magnitude.rank, 
                             tf_optimization = 'robust')
    ti.to_device('cpu') # ensures can be pickled properly
    tensors_individual[score_type] = ti
c2c.io.export_variable_with_pickle(tensors_individual, output_folder + '/BALF-Tensor_individualscores.pkl')

# tensors_individual = c2c.io.read_data.load_variable_with_pickle(output_folder + '/BALF-Tensor_individualscores.pkl')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:18<00:00,  1.54s/it]
/home/hratch/miniconda3/envs/ccc_protocols/lib/python3.11/site-packages/tensorly/backend/pytorch_backend.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  7%|█████████                                                                                                                        | 7/100 [00:25<05:38,  3.64s/it]

Next, we can calculate the similarity between each score type's deomposition output based on the CorrIndex metric. For more information on the CorrIndex, as well as similar analyses on decomposition consistency, see Figure 3A of this [paper](https://doi.org/10.1038/s41467-022-31369-2):

In [ ]:
corrindex_res = pd.DataFrame(columns = ['score_type_1', 'score_type_2', 'Similarity'])

# calculate corrindex pairwise between score types
for idx, score_type_comparison in enumerate(itertools.permutations(tensors_individual, 2)):
    tensor_1 = tensors_individual[score_type_comparison[0]]
    tensor_2 = tensors_individual[score_type_comparison[1]]
    corrindex = correlation_index(tensor_1.factors, tensor_2.factors)
    similarity = 1 - corrindex
    
    corrindex_res.loc[idx, :] = list(score_type_comparison) + [similarity]

# formatting
corrindex_res = corrindex_res.pivot(index='score_type_1', columns='score_type_2', values = 'Similarity')
corrindex_res.columns = corrindex_res.columns.tolist()
corrindex_res.index = corrindex_res.index.tolist()
np.fill_diagonal(corrindex_res.values, 1)
for col_name in corrindex_res.columns:
    corrindex_res[col_name] = corrindex_res[col_name].astype(float)

In [ ]:
cm = c2c.plotting.clustermap_cci(corrindex_res,
                                 method='ward',
                                 optimal_leaf=True,
                                 metadata=None,
                                 title='',
                                 cbar_title='Similarity', 
                                 cmap='Blues_r',
#                                  vmax=1.,
#                                  vmin=0.,
                                 annot=False, 
                                 dendrogram_ratio=0.15,
                                figsize=(4,5))

font = matplotlib.font_manager.FontProperties(weight='bold', size=7)
for ax in [cm.ax_heatmap, cm.ax_cbar]:
    for tick in ax.get_xticklabels():
        tick.set_fontproperties(font)
    for tick in ax.get_yticklabels():
        tick.set_fontproperties(font)

    text = ax.yaxis.label
    text.set_font_properties(font)

Given the high overall similarity between score types, we see that Tensor-cell2cell's decomposition tends to capture consistent patterns across samples, smoothing over some of the inconsistencies in communication scores that we saw at the individual sample level in [Tutorial 02](./02-Infer-Communication-Scores.ipynb) 